## Load Data

In [20]:
import pandas as pd
import numpy as np
imd_df = pd.read_csv('./data/Replica_processed/imputed-50-trips-end-at-imd-2024-spring-thursday.csv')
imd_df["sampno"] = np.arange(1, len(imd_df) + 1)
imd_df['active_cost'] = 0 
imd_df['active_dist'] = (imd_df['bike_dist'] + imd_df['walk_dist']) / 2
imd_df['bike_cost'] = 0 
imd_df['walk_cost'] = 0 
imd_df['auto_cost'] = imd_df['parking_cost'] 
imd_df['worker'] = (imd_df['purpose'] == 'work').astype(int)
imd_df['highincome'] = (imd_df['hhinc'] > 0.06).astype(int)

In [21]:
long_four_imd_df = pd.DataFrame()

modes = {
    0: "auto",
    1: "transit",
    2: "bike",
    3: "walk"
}

# Normalize selected columns
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalize_cols = [
    "auto_time", "auto_dist",'auto_cost',
    "transit_time", "transit_dist",'transit_cost',
    "bike_time", "bike_dist",'bike_cost',
    "walk_time", "walk_dist",'walk_cost',
    "age", "hhinc", "numvec", "hhsize",'worker'
]
imd_df[normalize_cols] = scaler.fit_transform(imd_df[normalize_cols])

# Generate long format
for mode_id, mode_name in modes.items():
    temp_data = pd.DataFrame({
        "sampno": imd_df["sampno"],
        "mode_four_kinds": mode_id,
        "choice": (imd_df["mode_four_kinds"] == mode_id).astype(int),
        "travel_time": imd_df[f"{mode_name}_time"],
        "travel_dist": imd_df[f"{mode_name}_dist"],
        "travel_cost": imd_df[f"{mode_name}_cost"],
        "age": imd_df["age"],
        "male": imd_df["male"],
        "hhinc": imd_df["hhinc"],
        "numvec": imd_df["numvec"],
        "hhsize": imd_df["hhsize"],
        "higheduc": imd_df["higheduc"],
        "worker": imd_df["worker"],
        'highincome': imd_df["highincome"],
    })
    long_four_imd_df = pd.concat([long_four_imd_df, temp_data], ignore_index=True)

long_four_imd_df = long_four_imd_df.sort_values(by=["sampno", "mode_four_kinds"]).reset_index(drop=True)
long_four_imd_df

,sampno,mode_four_kinds,choice,travel_time,travel_dist,travel_cost,age,male,hhinc,numvec,hhsize,higheduc,worker,highincome
0,1,0,1,0.294621,0.216613,0.210120,0.582418,0,0.102456,1.0,0.5000,0,0.0,1
1,1,1,0,0.331970,0.115466,0.130909,0.582418,0,0.102456,1.0,0.5000,0,0.0,1
2,1,2,0,0.133980,0.126619,0.000000,0.582418,0,0.102456,1.0,0.5000,0,0.0,1
3,1,3,0,0.153816,0.152826,0.000000,0.582418,0,0.102456,1.0,0.5000,0,0.0,1
4,2,0,1,0.244138,0.133463,0.210120,0.285714,1,0.027067,1.0,0.1250,1,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176719,44180,3,0,0.063707,0.073127,0.000000,0.285714,1,0.072922,1.0,0.3125,0,0.0,1
176720,44181,0,1,0.645241,0.514356,0.027330,0.494505,0,0.079821,1.0,0.1875,0,1.0,1
176721,44181,1,0,0.865065,0.575109,0.000000,0.494505,0,0.079821,1.0,0.1875,0,1.0,1
176722,44181,2,0,0.522993,0.510250,0.000000,0.494505,0,0.079821,1.0,0.1875,0,1.0,1


## MNL with Travel Time

In [ ]:
import pylogit as pl
from collections import OrderedDict

# reproducible sampling
np.random.seed(24)
unique_obs      = long_four_imd_df['sampno'].unique()
n_alts          = long_four_imd_df['mode_four_kinds'].nunique()
n_obs_to_sample = 100000 // n_alts
sampled_obs     = np.random.choice(unique_obs, n_obs_to_sample, replace=False)

df_sample = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled_obs)].copy()
df_sample.sort_values(['sampno', 'mode_four_kinds'], inplace=True)


spec = OrderedDict()
name = OrderedDict()

spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']
# spec['travel_cost'] = [0, 1]
# name['travel_cost'] = ['cost_auto', 'cost_transit']
# spec['travel_dist'] = [0, 1, 2, 3]
# name['travel_dist'] = ['dist_auto', 'dist_transit', 'dist_bike', 'dist_walk']

mnl = pl.create_choice_model(
    data=df_sample,  
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

mnl.fit_mle(np.zeros(7))
print(mnl.get_statsmodels_summary())

Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.57 seconds.
Final log-likelihood: -13,396.3241


In [ ]:
import pylogit as pl
from collections import OrderedDict

# reproducible sampling
np.random.seed(4)
unique_obs      = long_four_imd_df['sampno'].unique()
n_alts          = long_four_imd_df['mode_four_kinds'].nunique()
n_obs_to_sample = 100000 // n_alts
sampled_obs     = np.random.choice(unique_obs, n_obs_to_sample, replace=False)

df_sample = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled_obs)].copy()
df_sample.sort_values(['sampno', 'mode_four_kinds'], inplace=True)


spec = OrderedDict()
name = OrderedDict()

spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']
# spec['travel_cost'] = [0, 1]
# name['travel_cost'] = ['cost_auto', 'cost_transit']
# spec['travel_dist'] = [0, 1, 2, 3]
# name['travel_dist'] = ['dist_auto', 'dist_transit', 'dist_bike', 'dist_walk']

mnl = pl.create_choice_model(
    data=df_sample,  
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

mnl.fit_mle(np.zeros(7))
print(mnl.get_statsmodels_summary())

Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.25 seconds.
Final log-likelihood: -13,515.2298
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,993
Method:                                MLE   Df Model:                            7
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.610
Time:                             13:07:36   Pseudo R-bar-squ.:               0.610
AIC:                            27,044.460   Log-Likelihood:            -13,515.230
BIC:                            27,101.346   LL-Null:                   -34,657.359
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
asc_transit     -1.8693      0.041    -45.158      0.000      -1.950      -1.78

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

## Add Travel Cost for MNL

In [ ]:
import pylogit as pl
from collections import OrderedDict

# reproducible sampling
np.random.seed(48)
unique_obs      = long_four_imd_df['sampno'].unique()
n_alts          = long_four_imd_df['mode_four_kinds'].nunique()
n_obs_to_sample = 100000 // n_alts
sampled_obs     = np.random.choice(unique_obs, n_obs_to_sample, replace=False)

df_sample = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled_obs)].copy()
df_sample.sort_values(['sampno', 'mode_four_kinds'], inplace=True)


spec = OrderedDict()
name = OrderedDict()

spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']
spec['travel_cost'] = [0, 1]
name['travel_cost'] = ['cost_auto', 'cost_transit']
# spec['travel_dist'] = [0, 1, 2, 3]
# name['travel_dist'] = ['dist_auto', 'dist_transit', 'dist_bike', 'dist_walk']

mnl = pl.create_choice_model(
    data=df_sample,  
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

mnl.fit_mle(np.zeros(9))
print(mnl.get_statsmodels_summary())

Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.29 seconds.
Final log-likelihood: -13,519.3614
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,991
Method:                                MLE   Df Model:                            9
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.610
Time:                             13:07:51   Pseudo R-bar-squ.:               0.610
AIC:                            27,056.723   Log-Likelihood:            -13,519.361
BIC:                            27,129.863   LL-Null:                   -34,657.359
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
asc_transit     -2.1154      0.062    -34.232      0.000      -2.237      -1.99

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

In [ ]:
import pylogit as pl
from collections import OrderedDict

# reproducible sampling
np.random.seed(37)
unique_obs      = long_four_imd_df['sampno'].unique()
n_alts          = long_four_imd_df['mode_four_kinds'].nunique()
n_obs_to_sample = 100000 // n_alts
sampled_obs     = np.random.choice(unique_obs, n_obs_to_sample, replace=False)

df_sample = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled_obs)].copy()
df_sample.sort_values(['sampno', 'mode_four_kinds'], inplace=True)


spec = OrderedDict()
name = OrderedDict()

spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']
spec['travel_cost'] = [0, 1]
name['travel_cost'] = ['cost_auto', 'cost_transit']
# spec['travel_dist'] = [0, 1, 2, 3]
# name['travel_dist'] = ['dist_auto', 'dist_transit', 'dist_bike', 'dist_walk']

mnl = pl.create_choice_model(
    data=df_sample,  
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

mnl.fit_mle(np.zeros(9))
print(mnl.get_statsmodels_summary())

Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.41 seconds.
Final log-likelihood: -13,392.3355
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,991
Method:                                MLE   Df Model:                            9
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.614
Time:                             13:08:07   Pseudo R-bar-squ.:               0.613
AIC:                            26,802.671   Log-Likelihood:            -13,392.336
BIC:                            26,875.811   LL-Null:                   -34,657.359
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
asc_transit     -2.1869      0.063    -34.460      0.000      -2.311      -2.06

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

In [ ]:
import pylogit as pl
from collections import OrderedDict

# reproducible sampling
np.random.seed(89)
unique_obs      = long_four_imd_df['sampno'].unique()
n_alts          = long_four_imd_df['mode_four_kinds'].nunique()
n_obs_to_sample = 100000 // n_alts
sampled_obs     = np.random.choice(unique_obs, n_obs_to_sample, replace=False)

df_sample = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled_obs)].copy()
df_sample.sort_values(['sampno', 'mode_four_kinds'], inplace=True)


spec = OrderedDict()
name = OrderedDict()

spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']
spec['travel_cost'] = [0, 1]
name['travel_cost'] = ['cost_auto', 'cost_transit']


mnl = pl.create_choice_model(
    data=df_sample,  
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

mnl.fit_mle(np.zeros(9))
print(mnl.get_statsmodels_summary())

Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.46 seconds.
Final log-likelihood: -13,505.0986
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,991
Method:                                MLE   Df Model:                            9
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.610
Time:                             13:08:22   Pseudo R-bar-squ.:               0.610
AIC:                            27,028.197   Log-Likelihood:            -13,505.099
BIC:                            27,101.337   LL-Null:                   -34,657.359
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
asc_transit     -2.1111      0.062    -33.979      0.000      -2.233      -1.98

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

## Add Social-demographic Variables for MNL

### add "age" to the model

In [ ]:
import numpy as np
import pylogit as pl
from collections import OrderedDict

# Sample a subset of observations
np.random.seed(24)
unique_obs = long_four_imd_df['sampno'].unique()
n_alts = long_four_imd_df['mode_four_kinds'].nunique()
n_obs = 100000 // n_alts
sampled = np.random.choice(unique_obs, n_obs, replace=False)
df = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled)].copy()
df.sort_values(['sampno', 'mode_four_kinds'], inplace=True)

# Model specification and names
spec = OrderedDict()
name = OrderedDict()

# Alternative-specific constants for modes 1,2,3 (mode 0 is base)
spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']

# Mode-specific travel time and cost
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']

# Socio-demographic variables: alt-specific for modes 0,2,3 only
person_vars = ['age']#, 'male', 'numvec', 'higheduc', 'hhsize', 'hhinc']
for var in person_vars:
    spec[var] = [0, 2, 3]
    name[var] = [f"{var}_auto", f"{var}_bike", f"{var}_walk"]

# Create and fit MNL model
mnl = pl.create_choice_model(
    data=df,
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

# Fit the model
n_params = sum(len(v) for v in spec.values())
init = np.zeros(n_params)
mnl.fit_mle(init)
print(mnl.get_statsmodels_summary())


Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.61 seconds.
Final log-likelihood: -13,324.8838
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,990
Method:                                MLE   Df Model:                           10
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.616
Time:                             13:08:38   Pseudo R-bar-squ.:               0.615
AIC:                            26,669.768   Log-Likelihood:            -13,324.884
BIC:                            26,751.034   LL-Null:                   -34,657.359
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
asc_transit     -1.3972      0.065    -21.564      0.000      -1.524      -1.27

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

### Add gender to the model

In [ ]:
import numpy as np
import pylogit as pl
from collections import OrderedDict

# Sample a subset of observations
np.random.seed(24)
unique_obs = long_four_imd_df['sampno'].unique()
n_alts = long_four_imd_df['mode_four_kinds'].nunique()
n_obs = 100000 // n_alts
sampled = np.random.choice(unique_obs, n_obs, replace=False)
df = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled)].copy()
df.sort_values(['sampno', 'mode_four_kinds'], inplace=True)

# Model specification and names
spec = OrderedDict()
name = OrderedDict()

# Alternative-specific constants for modes 0,2,3 (mode 0 is base)
spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']

# Mode-specific travel time and cost
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']

# Socio-demographic variables: alt-specific for modes 1,2,3 only
person_vars = ['male']#, 'numvec', 'higheduc', 'hhsize', 'hhinc']
for var in person_vars:
    spec[var] = [0, 2, 3]
    name[var] = [f"{var}_auto", f"{var}_bike", f"{var}_walk"]

# Create and fit MNL model
mnl = pl.create_choice_model(
    data=df,
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

# Fit the model
n_params = sum(len(v) for v in spec.values())
init = np.zeros(n_params)
mnl.fit_mle(init)
print(mnl.get_statsmodels_summary())


Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.56 seconds.
Final log-likelihood: -13,375.0500
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,990
Method:                                MLE   Df Model:                           10
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.614
Time:                             13:08:54   Pseudo R-bar-squ.:               0.614
AIC:                            26,770.100   Log-Likelihood:            -13,375.050
BIC:                            26,851.366   LL-Null:                   -34,657.359
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
asc_transit     -1.9054      0.046    -41.748      0.000      -1.995      -1.81

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

### add number of vehicles

In [ ]:


import numpy as np
import pylogit as pl
from collections import OrderedDict

# Sample a subset of observations
np.random.seed(24)
unique_obs = long_four_imd_df['sampno'].unique()
n_alts = long_four_imd_df['mode_four_kinds'].nunique()
n_obs = 100000 // n_alts
sampled = np.random.choice(unique_obs, n_obs, replace=False)
df = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled)].copy()
df.sort_values(['sampno', 'mode_four_kinds'], inplace=True)

# Model specification and names
spec = OrderedDict()
name = OrderedDict()

# Alternative-specific constants for modes 1,2,3 (mode 0 is base)
spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']

# Mode-specific travel time and cost
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']

# Socio-demographic variables: alt-specific for modes 1,2,3 only
person_vars = ['numvec']#, , 'higheduc', 'hhsize', 'hhinc']
for var in person_vars:
    spec[var] = [0, 2, 3]
    name[var] = [f"{var}_auto", f"{var}_bike", f"{var}_walk"]

# Create and fit MNL model
mnl = pl.create_choice_model(
    data=df,
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

# Fit the model
n_params = sum(len(v) for v in spec.values())
init = np.zeros(n_params)
mnl.fit_mle(init)
print(mnl.get_statsmodels_summary())


Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.37 seconds.
Final log-likelihood: -11,740.9675
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,990
Method:                                MLE   Df Model:                           10
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.661
Time:                             13:09:10   Pseudo R-bar-squ.:               0.661
AIC:                            23,501.935   Log-Likelihood:            -11,740.968
BIC:                            23,583.201   LL-Null:                   -34,657.359
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
asc_transit     -0.3905      0.052     -7.554      0.000      -0.492      -0.28

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

### add highedu

In [ ]:
import numpy as np
import pylogit as pl
from collections import OrderedDict

# Sample a subset of observations
np.random.seed(24)
unique_obs = long_four_imd_df['sampno'].unique()
n_alts = long_four_imd_df['mode_four_kinds'].nunique()
n_obs = 100000 // n_alts
sampled = np.random.choice(unique_obs, n_obs, replace=False)
df = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled)].copy()
df.sort_values(['sampno', 'mode_four_kinds'], inplace=True)

# Model specification and names
spec = OrderedDict()
name = OrderedDict()

# Alternative-specific constants for modes 1,2,3 (mode 0 is base)
spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']

# Mode-specific travel time and cost
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']

# Socio-demographic variables: alt-specific for modes 0,2,3 only
person_vars = ['higheduc']#, , , 'hhsize', 'hhinc']
for var in person_vars:
    spec[var] = [0, 2, 3]
    name[var] = [f"{var}_auto", f"{var}_bike", f"{var}_walk"]

# Create and fit MNL model
mnl = pl.create_choice_model(
    data=df,
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

# Fit the model
n_params = sum(len(v) for v in spec.values())
init = np.zeros(n_params)
mnl.fit_mle(init)
print(mnl.get_statsmodels_summary())


Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.57 seconds.
Final log-likelihood: -13,376.0146
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,990
Method:                                MLE   Df Model:                           10
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.614
Time:                             13:09:26   Pseudo R-bar-squ.:               0.614
AIC:                            26,772.029   Log-Likelihood:            -13,376.015
BIC:                            26,853.295   LL-Null:                   -34,657.359
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
asc_transit      -1.8683      0.055    -34.211      0.000      -1.975      -1

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

### Add household size

In [ ]:
import numpy as np
import pylogit as pl
from collections import OrderedDict

# Sample a subset of observations
np.random.seed(24)
unique_obs = long_four_imd_df['sampno'].unique()
n_alts = long_four_imd_df['mode_four_kinds'].nunique()
n_obs = 100000 // n_alts
sampled = np.random.choice(unique_obs, n_obs, replace=False)
df = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled)].copy()
df.sort_values(['sampno', 'mode_four_kinds'], inplace=True)

# Model specification and names
spec = OrderedDict()
name = OrderedDict()

# Alternative-specific constants for modes 1,2,3 (mode 0 is base)
spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']

# Mode-specific travel time and cost
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']

# Socio-demographic variables: alt-specific for modes 0,2,3 only
person_vars = ['hhsize']#, , , , 'hhinc']
for var in person_vars:
    spec[var] = [0, 2, 3]
    name[var] = [f"{var}_auto", f"{var}_bike", f"{var}_walk"]

# Create and fit MNL model
mnl = pl.create_choice_model(
    data=df,
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

# Fit the model
n_params = sum(len(v) for v in spec.values())
init = np.zeros(n_params)
mnl.fit_mle(init)
print(mnl.get_statsmodels_summary())


Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.62 seconds.
Final log-likelihood: -13,269.4219
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,990
Method:                                MLE   Df Model:                           10
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.617
Time:                             13:09:42   Pseudo R-bar-squ.:               0.617
AIC:                            26,558.844   Log-Likelihood:            -13,269.422
BIC:                            26,640.110   LL-Null:                   -34,657.359
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
asc_transit     -1.5754      0.047    -33.545      0.000      -1.667      -1.48

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

### Add household income

In [ ]:
import numpy as np
import pylogit as pl
from collections import OrderedDict

# Sample a subset of observations
np.random.seed(24)
unique_obs = long_four_imd_df['sampno'].unique()
n_alts = long_four_imd_df['mode_four_kinds'].nunique()
n_obs = 100000 // n_alts
sampled = np.random.choice(unique_obs, n_obs, replace=False)
df = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled)].copy()
df.sort_values(['sampno', 'mode_four_kinds'], inplace=True)

# Model specification and names
spec = OrderedDict()
name = OrderedDict()

# Alternative-specific constants for modes 1,2,3 (mode 0 is base)
spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']

# Mode-specific travel time and cost
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']

# Socio-demographic variables: alt-specific for modes 0,2,3 only
person_vars = ['hhinc']#, , , , ]
for var in person_vars:
    spec[var] = [0, 2, 3]
    name[var] = [f"{var}_auto", f"{var}_bike", f"{var}_walk"]

# Create and fit MNL model
mnl = pl.create_choice_model(
    data=df,
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

# Fit the model
n_params = sum(len(v) for v in spec.values())
init = np.zeros(n_params)
mnl.fit_mle(init)
print(mnl.get_statsmodels_summary())


Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.36 seconds.
Final log-likelihood: -13,337.2194
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,990
Method:                                MLE   Df Model:                           10
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.615
Time:                             13:10:01   Pseudo R-bar-squ.:               0.615
AIC:                            26,694.439   Log-Likelihood:            -13,337.219
BIC:                            26,775.705   LL-Null:                   -34,657.359
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
asc_transit     -1.6781      0.048    -35.308      0.000      -1.771      -1.58

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

## Full MNL Model

In [ ]:
import numpy as np
import pylogit as pl
from collections import OrderedDict

# Sample a subset of observations
np.random.seed(24)
unique_obs = long_four_imd_df['sampno'].unique()
n_alts = long_four_imd_df['mode_four_kinds'].nunique()
n_obs = 100000 // n_alts
sampled = np.random.choice(unique_obs, n_obs, replace=False)
df = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled)].copy()
df.sort_values(['sampno', 'mode_four_kinds'], inplace=True)

# Model specification and names
spec = OrderedDict()
name = OrderedDict()

# Alternative-specific constants for modes 1,2,3 (mode 0 is base)
spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']

# Mode-specific travel time and cost
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']


# Socio-demographic variables: alt-specific for modes 0,2,3 only
person_vars = ['age', 'male', 'numvec',  'hhinc']
for var in person_vars:
    spec[var] = [0, 2, 3]
    name[var] = [f"{var}_auto", f"{var}_bike", f"{var}_walk"]

# Create and fit MNL model
mnl = pl.create_choice_model(
    data=df,
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

# Fit the model
n_params = sum(len(v) for v in spec.values())
init = np.zeros(n_params)
mnl.fit_mle(init)
# Fit the model and capture the results object
res = mnl.fit_mle(init)
mnl.get_statsmodels_summary()

Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.91 seconds.
Final log-likelihood: -11,639.3875
Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.84 seconds.
Final log-likelihood: -11,639.3875


/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,981
Method:                                MLE   Df Model:                           19
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.664
Time:                             13:10:34   Pseudo R-bar-squ.:               0.664
AIC:                            23,316.775   Log-Likelihood:            -11,639.388
BIC:                            23,471.181   LL-Null:                   -34,657.359
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
asc_transit     -0.0345      0.078     -0.442      0.658      -0.188       0.118
asc_bike        -0.9672      0.158     -6.111      0.000      -1.277      -0.657
asc_walk         2.2553      0.100     22.623      0.000       2.060       2.451
time_auto       -3.3300      0.216    -15.387      0.000      -3.754      -2.906
time_transit    -3.0846      0.235    -13.144      0.000      -3.545      -2.625
time_bike      -13.9113      0.888    -15.671      0.000     -15.651     -12.171
time_walk      -78.3416      1.827    -42.874      0.000     -81.923     -74.760
age_auto         1.2594      0.136      9.292      0.000       0.994       1.525
age_bike        -0.1571      0.316     -0.497      0.619      -0.777       0.462
age_walk         0.3810      0.204      1.870      0.061      -0.018       0.780
male_auto       -0.1124      0.048     -2.356      0.018      -0.206      -0.019
male_bike        0.4355      0.107      4.062      0.000       0.225       0.646
male_walk        0.0348      0.071      0.492      0.622      -0.104       0.173
numvec_auto      4.2493      0.101     42.218      0.000       4.052       4.447
numvec_bike     -0.9646      0.244     -3.946      0.000      -1.444      -0.485
numvec_walk      1.6626      0.138     12.087      0.000       1.393       1.932
hhinc_auto      -1.3994      0.261     -5.353      0.000      -1.912      -0.887
hhinc_bike       0.3393      0.559      0.607      0.544      -0.757       1.436
hhinc_walk      -1.8919      0.409     -4.622      0.000      -2.694      -1.090
================================================================================
"""

In [ ]:
import numpy as np
import pylogit as pl
from collections import OrderedDict

# Sample a subset of observations
np.random.seed(24)
unique_obs = long_four_imd_df['sampno'].unique()
n_alts = long_four_imd_df['mode_four_kinds'].nunique()
n_obs = 100000 // n_alts
sampled = np.random.choice(unique_obs, n_obs, replace=False)
df = long_four_imd_df[long_four_imd_df['sampno'].isin(sampled)].copy()
df.sort_values(['sampno', 'mode_four_kinds'], inplace=True)

# Model specification and names
spec = OrderedDict()
name = OrderedDict()

# Alternative-specific constants for modes 1,2,3 (mode 0 is base)
spec['intercept'] = [1, 2, 3]
name['intercept'] = ['asc_transit', 'asc_bike', 'asc_walk']

# Mode-specific travel time and cost
spec['travel_time'] = [0, 1, 2, 3]
name['travel_time'] = ['time_auto', 'time_transit', 'time_bike', 'time_walk']

spec['travel_cost'] = [0, 1]
name['travel_cost'] = ['cost_auto', 'cost_transit']

# Socio-demographic variables: alt-specific for modes 0,2,3 only
person_vars = ['age', 'male', 'numvec',  'hhinc']
for var in person_vars:
    spec[var] = [0, 2, 3]
    name[var] = [f"{var}_auto", f"{var}_bike", f"{var}_walk"]

# Create and fit MNL model
mnl = pl.create_choice_model(
    data=df,
    alt_id_col='mode_four_kinds',
    obs_id_col='sampno',
    choice_col='choice',
    specification=spec,
    model_type='MNL',
    names=name
)

# Fit the model
n_params = sum(len(v) for v in spec.values())
init = np.zeros(n_params)
mnl.fit_mle(init)
mnl.get_statsmodels_summary()

Log-likelihood at zero: -34,657.3590
Initial Log-likelihood: -34,657.3590


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 1.00 seconds.
Final log-likelihood: -11,631.2315


/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:454: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params_data = lzip([forg(params[i], prec=4) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:455: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [forg(std_err[i]) for i in exog_idx],
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/iolib/summary.py:456: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a val

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      choice   No. Observations:               25,000
Model:             Multinomial Logit Model   Df Residuals:                   24,979
Method:                                MLE   Df Model:                           21
Date:                     Mon, 19 May 2025   Pseudo R-squ.:                   0.664
Time:                             13:10:51   Pseudo R-bar-squ.:               0.664
AIC:                            23,304.463   Log-Likelihood:            -11,631.232
BIC:                            23,475.122   LL-Null:                   -34,657.359
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
asc_transit     -0.2122      0.093     -2.283      0.022      -0.394      -0.030
asc_bike        -1.0883      0.162     -6.737      0.000      -1.405      -0.772
asc_walk         2.0875      0.109     19.123      0.000       1.874       2.301
time_auto       -3.3733      0.217    -15.552      0.000      -3.798      -2.948
time_transit    -2.9619      0.237    -12.517      0.000      -3.426      -2.498
time_bike      -13.6085      0.885    -15.380      0.000     -15.343     -11.874
time_walk      -76.1526      1.900    -40.085      0.000     -79.876     -72.429
cost_auto       -0.2486      0.071     -3.526      0.000      -0.387      -0.110
cost_transit     0.1158      0.081      1.438      0.151      -0.042       0.274
age_auto         1.2523      0.136      9.227      0.000       0.986       1.518
age_bike        -0.1735      0.316     -0.549      0.583      -0.793       0.446
age_walk         0.3496      0.204      1.714      0.087      -0.050       0.749
male_auto       -0.1168      0.048     -2.446      0.014      -0.210      -0.023
male_bike        0.4356      0.107      4.063      0.000       0.226       0.646
male_walk        0.0366      0.071      0.518      0.604      -0.102       0.175
numvec_auto      4.2502      0.102     41.849      0.000       4.051       4.449
numvec_bike     -0.9871      0.245     -4.035      0.000      -1.467      -0.508
numvec_walk      1.6266      0.138     11.793      0.000       1.356       1.897
hhinc_auto      -1.4068      0.263     -5.349      0.000      -1.922      -0.891
hhinc_bike       0.3843      0.559      0.688      0.492      -0.711       1.480
hhinc_walk      -1.8889      0.410     -4.605      0.000      -2.693      -1.085
================================================================================
"""

## Check Accessibility

In [ ]:
df

,sampno,mode_four_kinds,choice,travel_time,travel_dist,travel_cost,age,male,hhinc,numvec,hhsize,higheduc,worker,intercept
0,1,0,1,0.294621,0.216613,0.210120,0.582418,0,0.102456,1.000000,0.5000,0,0.0,1.0
1,1,1,0,0.331970,0.115466,0.130909,0.582418,0,0.102456,1.000000,0.5000,0,0.0,1.0
2,1,2,0,0.133980,0.126619,0.000000,0.582418,0,0.102456,1.000000,0.5000,0,0.0,1.0
3,1,3,0,0.153816,0.152826,0.000000,0.582418,0,0.102456,1.000000,0.5000,0,0.0,1.0
4,2,0,1,0.244138,0.133463,0.210120,0.285714,1,0.027067,1.000000,0.1250,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176711,44178,3,0,0.399041,0.392048,0.000000,0.659341,0,0.119123,0.666667,0.0625,1,0.0,1.0
176716,44180,0,1,0.191724,0.081885,0.198400,0.285714,1,0.072922,1.000000,0.3125,0,0.0,1.0
176717,44180,1,0,0.220547,0.066739,0.716364,0.285714,1,0.072922,1.000000,0.3125,0,0.0,1.0
176718,44180,2,0,0.075441,0.058177,0.000000,0.285714,1,0.072922,1.000000,0.3125,0,0.0,1.0


In [ ]:
from compute_accessibility import compute_accessibility
group_stats, full_df = compute_accessibility(df, mnl, group_col='worker')
group_stats

,mean,median,std
worker,,,
no,2.007971,1.974017,1.292594
yes,1.646107,1.564550,1.250025


In [ ]:
group_stats, full_df = compute_accessibility(df, mnl, group_col='higheduc')
group_stats

,mean,median,std
higheduc,,,
no,1.900181,1.87111,1.374751
yes,1.771870,1.69333,1.242174


In [ ]:
group_stats, full_df = compute_accessibility(df, mnl, group_col='highincome')
group_stats

KeyError: "['highincome'] not in index"